este programa tiene la intencion de limpiar la base de datos del c4, la forma de limpiar se hara:
    eliminando caractereres especiales en los nombres
    reduciendo nombres culla similitud sea mayor al 90% dentro del top
    se entrena un modelo para predecir a los 1000 mas buscados y despues buscar a los 1000 mas buscados en el resto de la data
    se ubican todos los que no poseen foto//falta
    se ubican todos los Nan//falta
    

In [1]:
#importar librerias
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
from fuzzywuzzy import fuzz
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.display import display


import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool



In [2]:
#setear carpetas
DATA1 = "/home/workstation/Desktop/fv/data/pics/pic1/"
DATA2 = "/home/workstation/Desktop/fv/data/pics/pic2/"
TOP_DATA = "/home/workstation/Desktop/fv/topwanted/"
ALL_DATA = "/home/workstation/Desktop/fv/allwanted/"
LOWCOUNT = "/home/workstation/Desktop/fv/lowcount/"
FACES = '/home/workstation/Desktop/fv/faces/'
FIT_DATA ="/home/workstation/Desktop/fv/fit/"
val_dir = "/home/workstation/Desktop/fv/fit/val/"
train_dir = "/home/workstation/Desktop/fv/fit/train/"
test_dir = "/home/workstation/Desktop/fv/fit/test/"

carpetas = [ DATA1, DATA2, TOP_DATA, ALL_DATA, LOWCOUNT, FACES, FIT_DATA,val_dir ,train_dir ,test_dir]

for carpeta in carpetas:
    if not os.path.exists(carpeta):
        os.mkdir(carpeta)
        
predict = '/home/workstation/Desktop/fv/perdictions/preds/'
crops = '/home/workstation/Desktop/fv/predictions/crops/'

In [3]:
#importamos DB
df_faces = pd.read_csv("/home/workstation/Desktop/fv/data/DB.csv",encoding="utf-8", index_col=0)

##creamos una lista de nombres
df_faces_nombre = df_faces.groupby(["nombre"])
max_names1 = len(df_faces_nombre)
print('numero de nombres diferentes:', max_names1)

/home/workstation/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


numero de nombres diferentes: 376885


In [4]:

from IPython.display import display
import ipywidgets as widgets


final_val = len(df_faces[df_faces['nombre'].str.contains("/")==True].index)+len(df_faces[df_faces['nombre'].str.contains(",")==True].index) + len(df_faces[df_faces['nombre'].str.contains("  ")==True].index) 
print(final_val)
progress = widgets.IntProgress(value=0, min=0, max=final_val)
display(progress)

k = 0
h = 0
l = 0

for index in df_faces[df_faces['nombre'].str.contains("/")==True].index:   
    df_faces.loc[index,'nombre'] = df_faces.loc[index,'nombre'].replace("/", " ")
    #salvar todos los que vamos modificando
    h +=1
    progress.value +=1
for index in df_faces[df_faces['nombre'].str.contains(",")==True].index:
    df_faces.loc[index,'nombre'] = df_faces.loc[index,'nombre'].replace(",", " ")
    progress.value +=1
    k +=1
for index in df_faces[df_faces['nombre'].str.contains("  ")==True].index:   
    df_faces.loc[index,'nombre'] = df_faces.loc[index,'nombre'].replace("  ", " ")
    l +=1
    progress.value +=1
Num_SpeciaChars = k+h+l
print(k, 'nombres con "/" +' , h, 'nombres con "," +',  '= ' , Num_SpeciaChars)


153669


IntProgress(value=0, max=153669)

70 nombres con "/" + 70 nombres con "," + =  153719


In [5]:
#creamos una lista de nombres
df_faces_nombre = df_faces.groupby(["nombre"])
max_names2 = len(df_faces_nombre)
print('numero de nombres diferentes:', max_names2)
reducto = max_names1-max_names2
print("se ha reducido en total:", reducto, "nombres")
print("en este paso se redujeron:", reducto)

numero de nombres diferentes: 358903
se ha reducido en total: 17982 nombres
en este paso se redujeron: 17982


In [8]:
#creamos una lista con los mil nombres mas comunes
df_faces_nombre = df_faces.groupby(["nombre"])
top_names = df_faces_nombre.count().sort_values(by=['foto1'], ascending=False).iloc[0:1300]["foto1"].index
#print(top_names)
data = pd.DataFrame(top_names)


In [9]:
from functools import partial
from fuzzywuzzy import fuzz
import multiprocessing as mp
import numpy as np

def fuzz_analisis0_partial(top_names, partial_top_names): 
    duplicates = [] 
    for name in top_names["nombre"]:
        for name2 in partial_top_names["nombre"]: 
            ratio = fuzz.ratio(name, name2)
            if ratio>95 and ratio<100: 
                duplicates.append((name, name2)) 
    return duplicates 

cores = mp.cpu_count() 
df_split = np.array_split(data, cores, axis=0) 

pool = mp.Pool(cores) 
for duplicates in pool.map(partial(fuzz_analisis0_partial, data), df_split): 
    for name, name2 in duplicates: 
        data[data["nombre"] == name2] = name
pool.close() 
pool.join()

In [10]:
print(len(data))
data = data.drop_duplicates()
print(len(data))
print(data)

1300
1243
                                   nombre
0                    VICTOR MORENO MORENO
1              SERGIO HERNANDEZ GUTIERREZ
2            FRANCISCO JAVIER MUÑOZ LOPEZ
3                     DANIEL OLVERA MUÑOZ
4               CESAR OMAR DELGADO SILVA 
...                                   ...
1295                          JUAN X LUGO
1296         LUIS ANGEL MARQUEZ RODRIGUEZ
1297                MANUEL RAMIREZ FLORES
1298  DAVID JEFFERSON GUTIERREZ HERNANDEZ
1299          DIMAS FABIAN MONJARAZ GOMEZ

[1243 rows x 1 columns]


In [11]:
def length_ratio(s1, s2):
    s1_len = len(s1)
    s2_len = len(s2)
    distance = s1_len - s2_len if s1_len > s2_len else s2_len - s1_len
    lensum = s1_len + s2_len
    return 100 - 100 * distance / lensum

def fuzz_analisis_partial(top_names, partial_top_names): 
    duplicates = [] 
    for name in data["nombre"]:
        for name2 in partial_top_names["nombre"]:   
            if length_ratio(name, name2) < 90:
                continue
            ratio = fuzz.ratio(name, name2)
            if ratio>95 and ratio<100: 
                duplicates.append((name, name2)) 
    return duplicates 

cores = mp.cpu_count() 
df_split = np.array_split(df_faces, cores, axis=0) 

pool = mp.Pool(cores) 
for duplicates in pool.map(partial(fuzz_analisis_partial, df_faces), df_split): 
    for name, name2 in duplicates: 
        df_faces["nombre"] = df_faces["nombre"].replace(name2, name) 
        

pool.close() 
pool.join()

In [12]:

df_faces_nombre = df_faces.groupby(["nombre"])
max_names3 = len(df_faces_nombre)
print('numero de nombres diferentes:', max_names3)
print('la base de datos se ha reducido en:', max_names1-max_names3, "nombres")
print('en este paso se redujo:', max_names1-max_names3-reducto)
#nota verificar que las personas en name2 y name son las mismas, con ayuda del outfuzz y el allwanted


numero de nombres diferentes: 357752
la base de datos se ha reducido en: 19133 nombres
en este paso se redujo: 1151


In [13]:
#exportamos la base de datos
export_csv = df_faces.to_csv(r'/home/workstation/Desktop/fv/data/DB_cleaned.csv',encoding="utf-8", header=True)

In [15]:
df_faces_nombre = df_faces.groupby(["nombre"])
top_names = df_faces_nombre.count().sort_values(by=['foto1'], ascending=False).iloc[0:1300]["foto1"].index
top_names

Index(['VICTOR MORENO MORENO', 'FRANCISCO JAVIER MUÑOZ LOPEZ',
       'SERGIO HERNANDEZ GUTIERREZ', 'CESAR OMAR DELGADO SILVA ',
       'JOSE GUADALUPE NEGRETE CAMACHO', 'JUAN NICOLAS FLORES MORALES',
       'DANIEL OLVERA MUÑOZ', 'IVAN ERNESTO SANCHEZ URROZ',
       'JUAN RAYMUNDO MORALES MARTINEZ', 'JOSE GUADALUPE GONZALEZ MENDEZ',
       ...
       'LUIS GERARDO MARQUEZ BARCO', 'JESUS CASTAÑEDA FALCON',
       'MARTIN PEREIRA REYES', 'JOSE DE JESUS PIÑA BERNAL',
       'JESUS ALBERTO RODRIGUEZ MARTINEZ', 'HECTOR EMMANUEL CORDOVA CAMARILLO',
       'JUAN DIEGO X GUTIERREZ', 'JOSE DE JESUS QUIROZ ANGUIANO',
       'JUAN RAFAEL MARTINEZ PEREZ', 'MARTIN SERRANO ORNELAS'],
      dtype='object', name='nombre', length=1300)